In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
import chromadb
import ollama
import os
from tqdm import tqdm
from langchain_chroma import Chroma
from langchain.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain.chains import RetrievalQA
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate

In [2]:
path=r"D:/HRAgent/Policies/ANTI-BRIBERY AND ANTI-CORRUPTION POLICY OF TATA INDUSTRIES.pdf"

In [3]:
loader=PyPDFLoader(path)

In [4]:
resultantpdf=[]

In [5]:
resultantpdf.extend(loader.load())

In [6]:
url="http://172.16.0.178:8010"

In [7]:
embedding=OllamaEmbeddings(model="mxbai-embed-large",base_url=url)

In [8]:
semanticchunker=SemanticChunker(embedding, breakpoint_threshold_type="percentile")

In [9]:
chunks=semanticchunker.create_documents([doc.page_content for doc in tqdm(resultantpdf)])

100%|████████████████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 14221.42it/s]


In [10]:
chroma_db = Chroma.from_documents(
    documents=chunks, 
    embedding=embedding, 
    persist_directory="knowledgebase", 
    collection_name="maxbai_embed_knolwdge"
)

In [11]:
chroma_db.as_retriever().get_relevant_documents("finance vice persident of tata insights and quants")

C:\Users\AdityaKumarSingh\AppData\Local\Temp\ipykernel_13276\4032791689.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chroma_db.as_retriever().get_relevant_documents("finance vice persident of tata insights and quants")


[Document(metadata={}, page_content='S. Sriram, Vice President (Finance) & \nCompany Secretary  (Email: ssriram@tata.com) has been designated as the  \nCompliance Officer . All reports, complaints, doubts or concerns in relation to this \nABAC Policy shall be raised by the Designated Persons to the Compliance Officer  or \nto the Company Ethics Counsellor. Every query or concern rais ed by any Designated \nPerson in relation to any suspected violation of this ABAC Policy shall be investigated \nby the Compliance Officer. All queries, concerns or complaints received by the Company Ethics Counsellor \ndealing with a bribe ry or corruption issue should be reported to the Compliance \nOfficer by the Company Ethics Counsellor. Any action required  to be undertaken \nunder this ABAC  Policy shall be taken by the Compliance Officer in accordance with \nthis ABAC Policy. The Compliance Officer shall have a functional reporting to the  \nDesignated Director , Dr. Nirmalya Kumar  (Email: nirmaly

In [12]:
prompt_template = """Use the following pieces of context to answer the question at the end. 
If you do not know the answer, please think rationally and answer from your own knowledge base.
Don't put extra information

{context}

Question: {question}
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [13]:
llm=ChatOllama(base_url=url,model="llama3.2:1b",temperature=0.1)

In [17]:
query="finance vp of tata iq"

In [18]:
chain_type_kwargs = {"prompt": PROMPT}
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", 
    retriever=chroma_db.as_retriever(), 
    chain_type_kwargs=chain_type_kwargs)

qa_chain.run(query)

'The Compliance Officer is responsible for investigating queries, concerns, or complaints raised by Designated Persons regarding suspected violations of the Anti-Bribery and Anti-Corruption Policy (ABAC Policy).'

In [16]:
chroma_db.as_retriever().get_relevant_documents("finance vp of tata iq?")

[Document(metadata={}, page_content='S. Sriram, Vice President (Finance) & \nCompany Secretary  (Email: ssriram@tata.com) has been designated as the  \nCompliance Officer . All reports, complaints, doubts or concerns in relation to this \nABAC Policy shall be raised by the Designated Persons to the Compliance Officer  or \nto the Company Ethics Counsellor. Every query or concern rais ed by any Designated \nPerson in relation to any suspected violation of this ABAC Policy shall be investigated \nby the Compliance Officer. All queries, concerns or complaints received by the Company Ethics Counsellor \ndealing with a bribe ry or corruption issue should be reported to the Compliance \nOfficer by the Company Ethics Counsellor. Any action required  to be undertaken \nunder this ABAC  Policy shall be taken by the Compliance Officer in accordance with \nthis ABAC Policy. The Compliance Officer shall have a functional reporting to the  \nDesignated Director , Dr. Nirmalya Kumar  (Email: nirmaly